In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pickle
import numpy as np

In [6]:
with open("/kaggle/input/chat-bot/train_qa.txt","rb") as  fp:
    train_data = pickle.load(fp)

In [7]:
with open("/kaggle/input/chat-bot/test_qa.txt","rb") as  fp:
    test_data = pickle.load(fp)

In [8]:
type(test_data)

list

In [9]:
type(train_data)

list

In [10]:
len(train_data)

10000

In [11]:
len(test_data)

1000

In [15]:
test_data[0][2]

'no'

In [20]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [21]:
vocab =set()

In [22]:
all_data = test_data + train_data

In [23]:
len(all_data)

11000

In [24]:
for story,question,answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [25]:
vocab.add("no")
vocab.add("yes")

In [26]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [27]:
len(vocab)

37

In [28]:
vocab_len = len(vocab) + 1

In [29]:
vocab_len

38

In [30]:
max_story_len = max([len(data[0]) for data in all_data])

In [64]:
max_question_len = max([len(data[1]) for data in all_data])

In [66]:
print(max_story_len)
print(max_question_len)

156
6


In [34]:
# Vectorizing the Data

In [35]:
vocab_size = len(vocab) + 1

In [42]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [44]:
# integer encode sequences of words


In [46]:
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [47]:
tokenizer.word_index

{'up': 1,
 'in': 2,
 'office': 3,
 'back': 4,
 'milk': 5,
 'went': 6,
 'left': 7,
 'dropped': 8,
 'bedroom': 9,
 'picked': 10,
 'no': 11,
 'garden': 12,
 'there': 13,
 'took': 14,
 'to': 15,
 'the': 16,
 'kitchen': 17,
 'travelled': 18,
 'put': 19,
 'down': 20,
 'grabbed': 21,
 '.': 22,
 'hallway': 23,
 'mary': 24,
 'john': 25,
 'got': 26,
 'sandra': 27,
 '?': 28,
 'discarded': 29,
 'yes': 30,
 'daniel': 31,
 'is': 32,
 'moved': 33,
 'apple': 34,
 'journeyed': 35,
 'bathroom': 36,
 'football': 37}

In [48]:
train_story_text = []
train_question_text = []
train_answers = []

for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)

In [51]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [55]:
# train_story_seq

In [54]:
len(train_story_seq)

10000

In [56]:
### Functionalize Vectorization

In [68]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len,max_question_len=max_question_len):
    '''
    INPUT: 
    
    data: consisting of Stories,Queries,and Answers
    word_index: word index dictionary from tokenizer
    max_story_len: the length of the longest story (used for pad_sequences function)
    max_question_len: length of the longest question (used for pad_sequences function)


    OUTPUT:
    
    Vectorizes the stories,questions, and answers into padded sequences. We first loop for every story, query , and
    answer in the data. Then we convert the raw words to an word index value. Then we append each set to their appropriate
    output list. Then once we have converted the words to numbers, we pad the sequences so they are all of equal length.
    
    Returns this in the form of a tuple (X,Xq,Y) (padded based on max lengths)
    '''
    
    
    # X = STORIES
    X = []
    # Xq = QUERY/QUESTION
    Xq = []
    # Y = CORRECT ANSWER
    Y = []
    
    
    for story, query, answer in data:
        
        # Grab the word index for every word in story
        x = [word_index[word.lower()] for word in story]
        # Grab the word index for every word in query
        xq = [word_index[word.lower()] for word in query]
        
        # Grab the Answers (either Yes/No so we don't need to use list comprehension here)
        # Index 0 is reserved so we're going to use + 1
        y = np.zeros(len(word_index) + 1)
        
        # Now that y is all zeros and we know its just Yes/No , we can use numpy logic to create this assignment
        #
        y[word_index[answer]] = 1
        
        # Append each set of story,query, and answer to their respective holding lists
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    # Finally, pad the sequences based on their max length so the RNN can be trained on uniformly long sequences.
        
    # RETURN TUPLE FOR UNPACKING
    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [69]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [70]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [72]:
inputs_test

array([[ 0,  0,  0, ..., 16,  9, 22],
       [ 0,  0,  0, ..., 16, 12, 22],
       [ 0,  0,  0, ..., 16, 12, 22],
       ...,
       [ 0,  0,  0, ..., 16, 34, 22],
       [ 0,  0,  0, ..., 16, 12, 22],
       [ 0,  0,  0, ..., 34, 13, 22]], dtype=int32)

In [73]:
queries_test

array([[32, 25,  2, 16, 17, 28],
       [32, 25,  2, 16, 17, 28],
       [32, 25,  2, 16, 12, 28],
       ...,
       [32, 24,  2, 16,  9, 28],
       [32, 27,  2, 16, 12, 28],
       [32, 24,  2, 16, 12, 28]], dtype=int32)

In [75]:
answers_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [76]:
sum(answers_test)

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
       503.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 497.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.])

In [77]:
tokenizer.word_index['yes']

30

In [78]:
tokenizer.word_index['no']

11